# Demo "Relaxed Releases with DB-Workloadtests": Bonus Tune All (SELECT) Queries for one table

## Disclaimer
THIS CODE AND INFORMATION ARE PROVIDED "AS IS" WITHOUT WARRANTY OF ANY KIND, EITHER EXPRESSED OR IMPLIED, INCLUDING BUT NOT LIMITED
TO THE IMPLIED WARRANTIES OF MERCHANTABILITY AND/OR FITNESS FOR
PARTICULAR PURPOSE. 

CODE IS WRITTEN FOR DEMO PURPOSES

(C) Martin Guth and others, 2020

## Prerequisites
* SQL Server 2008 or newer (tested with SQL Server 2019 on Linux resp Docker)
* [StackOverflow-Database](https://www.brentozar.com/archive/2015/10/how-to-download-the-stack-overflow-database-via-bittorrent/) (tested with StackOverflow2010 aka small edition)

## Collecting Queries from Plan Cache
The following statement belongs to the classics of performance tuning from the pre-query-store-era. Queries are collected from the DMV sys.dm_exec_query_stats. Via DMFs like dm_exec_sql_text() relevant details are added. The filtering for the relevant table is done via a LIKE-search in the query_text. Alternatively a specific index could be filtered via a LIKE-search in the query plan text.

Typically such statements don't qualify as top performers themselves and are known to run for minutes. The collected information is saved in a table for further analysis...let's again save this in the database "WorkloadAnalysis".

The following query selects all queries on the "Posts" table.






In [1]:
-- Run some queries just for testing purposes and to create something to select :-)
DECLARE @id INT = 2680968,
@userId INT = 177420,
@sql NVARCHAR(500) = N'SELECT * FROM StackOverflow2010.dbo.Posts WHERE id = @id AND OwnerUserId= @userId',
@sqlparm NVARCHAR(500) = '@id INTEGER, @userId INTEGER'


EXECUTE sp_executeSql
	@sql,
	@sqlparm,
	@id,
	@userId
	;



SELECT p.CreationDate FROM StackOverflow2010.dbo.Posts p INNER JOIN StackOverflow2010.dbo.Users u ON p.owneruserid = u.id WHERE u.id = 2608972;

(1 row affected)

(0 rows affected)

Total execution time: 00:00:00.063

Id AcceptedAnswerId AnswerCount Body ClosedDate CommentCount CommunityOwnedDate CreationDate FavoriteCount LastActivityDate LastEditDate LastEditorDisplayName LastEditorUserId OwnerUserId ParentId PostTypeId Score Tags Title ViewCount 2680968 0 0 <p>This is not a ruby-on-rails question. What you need is some javascript library to do that. As mentioned by Salil, lightbox is one such library. Others include:</p>

<ul>
<li><a href="http://colorpowered.com/colorbox/" rel="nofollow noreferrer">colorbox</a></li>
<li><a href="http://jqueryui.com/demos/dialog/" rel="nofollow noreferrer">jQuery UI dialog</a></li>
<li><a href="http://fancybox.net/" rel="nofollow noreferrer">Fancybox</a></li>
<li><a href="http://www.shadowbox-js.com/index.html" rel="nofollow noreferrer">shadowbox</a></li>
</ul>

<p>I personally tried lightbox, thickbox (no longer supported), colorbox, and shadowbox, and found colorbox to best fit my needs.</p>
 NULL 1 NULL 2010-04-21 07:16:20.267 0 2010-04-21 07:16:20.267 NULL NULL 0 177420 2680803 2 2 NULL NULL 0

CreationDate

In [2]:
-- Code stolen from Kendra Little, Glen Berry and who nows else :-)
USE WorkloadAnalysis;
DROP TABLE IF EXISTS dbo.posts_20200511;
SELECT
	ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) AS id, 
	qs.query_hash, --Are these all different queries?
	qs.query_plan_hash, --Do they each have their own plan?
	SUBSTRING(tx.[text],
		(qs.statement_start_offset / 2) + 1,
		(CASE WHEN qs.statement_end_offset =-1 THEN DATALENGTH(tx.text) ELSE qs.statement_end_offset END - qs.statement_start_offset)
		/ 2 + 1) AS QueryText,
	case when pl.query_plan LIKE '%<MissingIndexes>%' then 1 else 0 end as [Missing Indexes?],
    qs.execution_count,
	qs.last_elapsed_time/1000	 AS last_elapsed_time,
	qs.total_elapsed_time/execution_count/1000 AS avg_elapsed_time,
	qs.total_elapsed_time/1000 AS total_elapsed_time,
	qs.last_worker_time/1000 AS last_cpu_time,
	qs.total_worker_time/execution_count/1000 AS avg_cpu_time,
	qs.total_worker_time/1000 AS total_cpu_time,
	qs.last_logical_reads,
	qs.total_logical_reads/execution_count AS avg_logical_reads,
	qs.total_logical_reads,
	qs.last_logical_reads / 128.0		AS last_logical_reads_MB,
	qs.creation_time AS [plan creation time],
	qs.last_execution_time [last execution time],
	plx.query_plan AS  sqlplan
INTO
	dbo.posts_20200511
FROM    sys.dm_exec_query_stats AS qs
        CROSS APPLY sys.dm_exec_text_query_plan(qs.plan_handle, qs.statement_start_offset, qs.statement_end_offset) AS pl
		CROSS APPLY sys.dm_exec_query_plan(qs.plan_handle) AS plx
        CROSS APPLY sys.dm_exec_sql_text(qs.sql_handle) AS tx
WHERE 
	SUBSTRING(tx.[text],
		(qs.statement_start_offset / 2) + 1,
		(CASE WHEN qs.statement_end_offset =-1 THEN DATALENGTH(tx.text) ELSE qs.statement_end_offset END - qs.statement_start_offset)
		/ 2 + 1) LIKE '%Posts%'
ORDER BY execution_count DESC OPTION (RECOMPILE);



(6 rows affected)

Total execution time: 00:00:05.225

The following statement prepares all queries for testing. Here only different queries are selected and a comment including some execution statistics is added. Additionally this code cares about parameters and sets them accordingly (herefore we need to concatenate strings via STRING_AGG...if you are running SQL lower than 2016 you could try some [CLR](https://www.mssqltips.com/sqlservertip/2022/concat-aggregates-sql-server-clr-function/) ).


In [3]:
SELECT		
	id,
	query_hash,
	hasParameters,
	'GO
	/* 
		Execution count      : ' + CAST(execution_count AS VARCHAR(50)) + '
		Last logical reads MB: ' + CAST(last_logical_reads_MB AS VARCHAR(50)) + '
		Last logical reads   : ' + CAST(last_logical_reads AS VARCHAR(50)) + ' 
		Avg  logical reads   : ' + CAST(avg_logical_reads AS VARCHAR(50)) + ' 
		Avg  elapsed time    : ' + CAST(avg_elapsed_time AS VARCHAR(50)) + '
		Avg  CPU     time    : ' + CAST(avg_cpu_time AS VARCHAR(50)) + '
	*/
	' + 
	CASE
		WHEN hasParameters = 'Y' THEN
            COALESCE(STRING_AGG('DECLARE ' + parameter + ' VARCHAR(255) = ' + value, ';'), '')
		ELSE
			''
	END 
	+ QueryText AS completeQueryText
FROM    
	(

		SELECT
			a.hasParameters,
			a.execution_count,
			a.id,
			a.query_hash,
			a.last_logical_reads_MB,
			a.last_logical_reads,
			a.avg_logical_reads,
			a.avg_elapsed_time,
			a.avg_cpu_time,
			a.QueryText,
			t.x.query('.').value('(/*:ColumnReference/@Column)[1]', 'varchar(20)') AS parameter,
			t.x.query('.').value('(/*:ColumnReference/@ParameterCompiledValue)[1]', 'varchar(200)') AS value
		FROM    
			(
				SELECT *
				FROM
				(
					SELECT  
						ROW_NUMBER() OVER (PARTITION BY query_hash ORDER BY LEN(QueryText) DESC) AS rownum,
						CASE 
							WHEN QueryText LIKE '%@%' THEN 
								'Y'
							ELSE 
								'N'
						END AS hasParameters,
						*
					FROM
						WorkloadAnalysis.dbo.posts_20200511
					WHERE	QueryText NOT LIKE 'UPDATE%' 
						AND QueryText NOT LIKE 'INSERT%'
						AND QueryText NOT LIKE 'DELETE%'
						AND QueryText NOT LIKE 'SELECT			id,	query_hash,	hasParameters' --die eigene Monitoringabfrage nicht ausgeben
				) itc
				WHERE
					rownum = 1
			) a
		OUTER APPLY sqlplan.nodes('//*:QueryPlan/*:ParameterList/*:ColumnReference') AS t(x)
	) src
GROUP BY
	id,
	query_hash,
	hasParameters,
	execution_count,
	last_logical_reads_MB,
	last_logical_reads,
	avg_logical_reads,
	avg_elapsed_time,
	avg_cpu_time,
	QueryText 
ORDER BY
	query_hash,
	execution_count,
	avg_logical_reads

(3 rows affected)

Total execution time: 00:00:00.222

id query_hash hasParameters completeQueryText 3 0x738C2958B0C85382 Y GO
	/* 
 Execution count : 5
 Last logical reads MB: 6283.078125
 Last logical reads : 804234 
 Avg logical reads : 804074 
 Avg elapsed time : 5626
 Avg CPU time : 24410
	*/
	DECLARE @count INT = (SELECT COUNT(*) FROM dbo.Posts WHERE CreationDate > '20200512') + 1 5 0xB77031979BBD7C1E Y GO
	/* 
 Execution count : 1
 Last logical reads MB: 0.031250
 Last logical reads : 4 
 Avg logical reads : 4 
 Avg elapsed time : 0
 Avg CPU time : 0
	*/
	DECLARE @userId VARCHAR(255) = (177420);DECLARE @id VARCHAR(255) = (2680968)SELECT * FROM StackOverflow2010.dbo.Posts WHERE id = @id AND OwnerUserId= @userId 6 0xBDEA76CCDA7D7C31 N GO
	/* 
 Execution count : 1
 Last logical reads MB: 0.023437
 Last logical reads : 3 
 Avg logical reads : 3 
 Avg elapsed time : 0
 Avg CPU time : 0
	*/
	SELECT p.CreationDate FROM StackOverflow2010.dbo.Posts p INNER JOIN StackOverflow2010.dbo.Users u ON p.owneruserid = u.id WHERE u.id = 2608972

1. Add the code from column "completeQueryText" and copy it to a new sql file.
1. Go to your test instance, run it and note runtime results (SET STATISTICS IO ON, SET STATISTICS TIME ON)
http://statisticsparser.com/ is recommended
1. Introduce your change (different indexes, code changes)
1. Run again
1. Note runtime results again
1. Document everything and make decision if change is good to introduce or not

This is a lot of manual labour after all...however it can pay off.

Here's an example for the generated code:
```

GO	
/* 		Execution count      : 2		Last logical reads MB: 0.031250		Last logical reads   : 4 		Avg  logical reads   : 4 		Avg  elapsed time    : 0		Avg  CPU     time    : 0	*/	
DECLARE @userId VARCHAR(255) = (177420);
DECLARE @id VARCHAR(255) = (2680968)
SELECT * FROM StackOverflow2010.dbo.Posts WHERE id = @id AND OwnerUserId= @userId
GO	
/* 		Execution count      : 1		Last logical reads MB: 0.023437		Last logical reads   : 3 		Avg  logical reads   : 3 		Avg  elapsed time    : 8		Avg  CPU     time    : 0	*/	
SELECT p.CreationDate FROM StackOverflow2010.dbo.Posts p INNER JOIN StackOverflow2010.dbo.Users u ON p.owneruserid = u.id WHERE u.id = 2608972
```